In [27]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from ipywidgets import widgets, interact, Dropdown, fixed
from IPython.display import display
import plotly.express as px
from datetime import datetime
import plotly.graph_objects as go

In [2]:
# Path to go up one level from the current folder to the parent folder
parent_path = os.path.join(os.getcwd(), '..')

# Path to the target data folder
data_folder_path = os.path.join(parent_path, 'Data')

In [3]:
menu_df = pd.read_csv(os.path.join(data_folder_path, 'menu_analysis.csv'))
customers_df = pd.read_csv(os.path.join(data_folder_path, 'customers.csv'))
stores_df = pd.read_csv(os.path.join(data_folder_path, 'stores.csv'))
structure_df = pd.read_csv(os.path.join(data_folder_path, 'structure.csv'))
size_df = pd.read_csv(os.path.join(data_folder_path, 'order_size.csv'))
transactions_df = pd.read_csv(
        os.path.join(data_folder_path, "transactions.csv"),
        parse_dates=['Transaction_Date']  # Modify as needed if format issues arise
    )

### Modification 1

In [4]:
# Assuming transactions_df has columns: 'Date', 'Store_ID', 'Total_Amount'
transactions_df['Transaction_Date'] = pd.to_datetime(transactions_df['Transaction_Date'])

### Modification 2

In [19]:
working_df = transactions_df.groupby(['Order_Number', 'Transaction_Date', 'Transaction_Time', 'Store_Number', 'Customer_ID'])['Price'].sum().reset_index()
working_df.head()

,Order_Number,Transaction_Date,Transaction_Time,Store_Number,Customer_ID,Price
0,1000,2024-01-09,16:59:33,STR_4,CUST6467,11.5
1,1001,2024-03-19,22:45:53,STR_9,CUST7534,27.0
2,1002,2023-07-14,13:06:28,STR_9,CUST5791,8.0
3,1003,2023-07-07,06:24:28,STR_8,CUST2406,6.5
4,1007,2023-06-21,13:47:10,STR_7,CUST6394,31.5


### Modification 3

In [22]:
# Calculate average transaction value by store and date
avg_transaction = working_df.groupby(['Transaction_Date', 'Store_Number'])['Price'].mean().reset_index()


In [23]:
avg_transaction.head()

,Transaction_Date,Store_Number,Price
0,2023-06-21,STR_1,15.068182
1,2023-06-21,STR_10,12.740741
2,2023-06-21,STR_2,16.354839
3,2023-06-21,STR_3,14.571429
4,2023-06-21,STR_4,13.913793


# Average Transaction Value by Store

In [26]:
# Create the plot
fig = go.Figure()

for store in avg_transaction['Store_Number'].unique():
    store_data = avg_transaction[avg_transaction['Store_Number'] == store]
    fig.add_trace(go.Scatter(
        x=store_data['Transaction_Date'],
        y=store_data['Price'],
        mode='lines',
        name=f'Store {store}',
        hovertemplate='<b>Store %{name}</b><br>Date: %{x|%Y-%m-%d}<br>Avg Transaction: $%{y:.2f}<extra></extra>'
    ))

fig.update_layout(
    title='Average Transaction Value by Store Over Time',
    xaxis_title='Date',
    yaxis_title='Average Transaction Value ($)',
    legend_title='Store',
    hovermode='closest'
)

# Add range slider
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=7, label="1w", step="day", stepmode="backward"),
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(visible=True),
        type="date"
    )
)

# Add annotations for overall average lines
overall_avg = transactions_df.groupby('Store_Number')['Price'].mean()
for store, avg in overall_avg.items():
    fig.add_shape(
        type="line",
        x0=transactions_df['Transaction_Date'].min(),
        x1=transactions_df['Transaction_Date'].max(),
        y0=avg,
        y1=avg,
        line=dict(color="rgba(0,0,0,0.3)", width=2, dash="dash"),
    )
    fig.add_annotation(
        x=transactions_df['Transaction_Date'].max(),
        y=avg,
        text=f"Store {store} Avg: ${avg:.2f}",
        showarrow=False,
        yshift=10
    )

fig.show()